# Create Annotations

This program creates an xml file and copy of an image for training machine learning models.
This program opens a directory and for each image in the folder:
- displays it
- Allows the user to drag the mouse to create rectangles of the areas in the image that contain items to train the model on.
- press w to write the xml file and image to go to the next image
- press + to go to the next image if there is nothing to train in the present image
-   "   1 to skip ahead 10 images
- press - to go to the previous image
-   "   e to end the program


In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install opencv-python

In [1]:
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_path,org_f_width,org_f_height,org_f_depth, refPts):
    # With credit to: 
    # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+obj_f_num+".jpg</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>pompeiiinpictures.com</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n") 
    annot_file.write("	<segmented>0</segmented>\n") 
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        #annot_file.write("		<pose>Unspecified</pose>\n") 
        annot_file.write("		<truncated>0</truncated>\n") 
        annot_file.write("		<difficult>0</difficult>\n") 
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close() 

In [2]:
# import the necessary packages

import cv2

# initialize the list of reference points

refPt = []
refPts = []

def click_and_crop(event, x, y, flags, param):
	# grab references to the global variables
	global refPt, cropping

	# if the left mouse button was clicked, record the starting
	# (x, y) coordinates and indicate that cropping is being
	# performed
	if event == cv2.EVENT_LBUTTONDOWN:
		refPt = [(x, y)]
		cropping = True

	# check to see if the left mouse button was released
	elif event == cv2.EVENT_LBUTTONUP:
		# record the ending (x, y) coordinates and indicate that
		# the cropping operation is finished
		refPt.append((x, y))
		cropping = False

		# draw a rectangle around the region of interest
		cv2.rectangle(resized, refPt[0], refPt[1], (0, 0, 255), 2)
		cv2.imshow("image", resized)
		refPts.append(refPt)

# Skip 

In [3]:
# With credit to:
# Adrian Rosebrock 
# https://www.pyimagesearch.com/2015/03/09/capturing-mouse-click-events-with-python-and-opencv/

# load the image, clone it, and setup the mouse callback function

refPts = []
original_file_path = '/home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10405 May 2017 DSCN1949.JPG'
image = cv2.imread(original_file_path)
print('Original Dimensions : ',image.shape)

# With credit to for image resizing
# https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
scale_percent = 1024/image.shape[1] # percent of original size
print(scale_percent)
width = int(image.shape[1] * scale_percent )
height = int(image.shape[0] * scale_percent )
dim = (width, height)
# resize image
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
resized_original = resized.copy()
 
print('Resized Dimensions : ',resized.shape)
 
#clone = resized.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

# keep looping until the 'c' key is pressed
while True:
	# display the image and wait for a keypress
	cv2.imshow("image", resized)
	key = cv2.waitKey(1) & 0xFF

	# if the 'c' key is pressed, break from the loop
	if key == ord("c"):
		break

print(refPts)

# close all open windows
cv2.destroyAllWindows()

object_type = 'opus_incertum'
object_annots_dir = '/home/student/data_5000_project/data/images/construction_types/processed/test/annots/'
object_images_dir = '/home/student/data_5000_project/data/images/construction_types/processed/test/images/'
object_file_num = '00001'
write_annot(object_type, object_annots_dir, object_file_num, original_file_path,resized.shape[1],resized.shape[0],resized.shape[2], refPts)
cv2.imwrite(object_images_dir+object_file_num+".jpg",resized_original)

Original Dimensions :  (3456, 4608, 3)
0.2222222222222222
Resized Dimensions :  (768, 1024, 3)
[]


True

In [13]:
def annotate_image(original_image_path, object_type, object_annots_dir, object_images_dir, object_file_num_int):
    global image,resized,resized_original,refPts
    # With credit to:
    # Adrian Rosebrock 
    # https://www.pyimagesearch.com/2015/03/09/capturing-mouse-click-events-with-python-and-opencv/
    object_file_num_int_return = object_file_num_int
    object_file_num = "0000"+str(object_file_num_int)
    object_file_num = object_file_num[-5:]
    print("original_image_path", original_image_path)
    # load the image, clone it, and setup the mouse callback function
    refPts = []
    image = []
    image = cv2.imread(original_image_path)
    #print('Original Dimensions 2: ',image.shape)
    #plt.imshow(imgage, cmap = 'gray', interpolation = 'bicubic')
    #plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    #plt.show()

    # With credit to for image resizing
    # https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
    scale_percent = 1024/image.shape[1] # percent of original size
    #print(scale_percent)
    width = int(image.shape[1] * scale_percent )
    height = int(image.shape[0] * scale_percent )
    dim = (width, height)
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    resized_original = resized.copy()
     
    #print('Resized Dimensions : ',resized.shape)
 
    #clone = resized.copy()
    #cv2.startWindowThread()
    #cv2.namedWindow("image")
    #cv2.setMouseCallback("image", click_and_crop)

    # keep looping until the 'c' key is pressed
    while True:
        # display the image and wait for a keypress
        cv2.imshow("image", resized)
        cv2.moveWindow("image", 0,100)
        key = cv2.waitKey(1) & 0xFF

        # if the 'c' key is pressed, break from the loop
        if key == ord("c"):
            object_file_num_int_return = object_file_num_int_return +1
            break
        if key == ord("w"):
            write_annot(object_type, object_annots_dir, object_file_num, original_image_path,resized.shape[1],resized.shape[0],resized.shape[2], refPts)
            cv2.imwrite(object_images_dir+object_file_num+".jpg",resized_original)
            object_file_num_int_return = object_file_num_int_return +1
            break
        if key == ord("-"):
            object_file_num_int_return = object_file_num_int_return-1
            break
        if key == ord("+"):
            object_file_num_int_return = object_file_num_int_return+1
            break
        if key == ord("1"):
            object_file_num_int_return = object_file_num_int_return+10
            break
        if key == ord("9"):
            object_file_num_int_return = object_file_num_int_return+90
            break            
        if key == ord("e"):
            object_file_num_int_return = -1
            break    

    #print(refPts)

    # close all open windows
    cv2.destroyAllWindows()
    image=None

    return object_file_num_int_return 
#object_type = 'opus_incertum'
#object_annots_dir = '/home/student/data_5000_project/data/images/construction_types/processed/annots/'
#object_images_dir = '/home/student/data_5000_project/data/images/construction_types/processed/images/'
#object_file_num = '00001'
    #write_annot(object_type, object_annots_dir, object_file_num, original_file_path,resized.shape[1],resized.shape[0],resized.shape[2], refPts)
    #cv2.imwrite(object_images_dir+object_file_num+".jpg",resized_original)

In [14]:
# With credit to for getting just files
# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory


raw_images_directory = '/home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/'
from os import listdir
from os.path import isfile, join
raw_image_files = [f for f in listdir(raw_images_directory) if isfile(join(raw_images_directory, f))]
object_type = 'opus_incertum'
object_annots_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/annots/'
object_images_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/images/'
object_file_num_int=0
while object_file_num_int>-1:
    cv2.startWindowThread()
    #cv2_window_name = "image" + str(object_file_num_int)+" "+raw_image_files[object_file_num_int]
    cv2_window_name = "image"
    cv2.namedWindow(cv2_window_name)
    cv2.setMouseCallback(cv2_window_name, click_and_crop)
    original_image_path = raw_images_directory + raw_image_files[object_file_num_int]
    #print(original_image_path)
    #print(raw_image_files[object_file_num_int])
    object_file_num_int = annotate_image(original_image_path, object_type, object_annots_dir, object_images_dir, object_file_num_int)
    print("                                                                                                  ",object_file_num_int)
    cv2.destroyAllWindows()

original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 11503 May 2015 DSCN2099.JPG
                                                                                                   1
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20904 May 2018 DSCN0218.JPG
                                                                                                   2
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 11004 May 2012 P1130436.JPG
                                                                                                   3
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20600 May 2016 DSCN8322.JPG
                                                                             

original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20806 May 2017 DSCN0097.JPG
                                                                                                   387
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 11413-14 May 2018 DSCN0133.JPG
                                                                                                   388
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20202 May 2016 DSCN8495.JPG
                                                                                                   389
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10904 May 2015 DSCN1989.JPG
                                                                    

                                                                                                   419
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10711 May 2017 DSCN1916.JPG
                                                                                                   420
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 12106 May 2016 DSCN8147.JPG
                                                                                                   421
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 11004 May 2012 P1130463.JPG
                                                                                                   422
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Fere

                                                                                                   441
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10711 May 2017 DSCN1930.JPG
                                                                                                   442
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20904 May 2018 DSCN0262.JPG
                                                                                                   443
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10711 May 2017 DSCN1915.JPG
                                                                                                   444
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Fere

                                                                                                   473
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10607 May 2016 DSCN8079.JPG
                                                                                                   474
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10805 May 2015 DSCN2164.JPG
                                                                                                   475
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10607 May 2016 DSCN8121.JPG
                                                                                                   476
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Fere

                                                                                                   505
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10905 May 2017 DSCN0636.JPG
                                                                                                   506
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10711 May 2017 DSCN0997.JPG
                                                                                                   507
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20202 May 2011 P1050532.JPG
                                                                                                   508
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Fere

                                                                                                   537
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10711 May 2017 DSCN0912.JPG
                                                                                                   538
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20700 May 2012 P1130235.JPG
                                                                                                   539
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20600 May 2018 DSCN0370.JPG
                                                                                                   540
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Fere

                                                                                                   569
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 20303 May 2016 DSCN7542.JPG
                                                                                                   570
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 12106 May 2016 DSCN8141.JPG
                                                                                                   571
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10602 May 2016 DSCN7150.JPG
                                                                                                   572
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Fere

                                                                                                   601
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10607 May 2016 DSCN8120.JPG
                                                                                                   602
original_image_path /home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10615 May 2015 DSCN4306.JPG
                                                                                                   -1


In [ ]:
print(onlyfiles[0] )

# Copy boxes for use by Osama

In [30]:
object_type = 'opus_incertum'
object_annots_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/annots/'
object_images_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/images/'
object_boxes_output_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/selected_training/'

import cv2
from xml.etree import ElementTree
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(object_annots_dir) if isfile(join(object_annots_dir, f))]

for fn in onlyfiles:
    #print(fn)
    tree = ElementTree.parse(object_annots_dir+fn)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    
    fn_image = root.find('./filename').text
    #print(fn_image)
    box_num=0
    for box in root.findall('.//bndbox'):
        box_num=box_num+1
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        #print(coors)
        
        img = cv2.imread(object_images_dir+fn_image)
        crop_img = img[ymin:ymax, xmin:xmax]
        print(crop_img.shape)
        fntd = object_boxes_output_dir+"td_"+fn[:5]+"_b"+str(box_num)+fn_image[5:]
        print(fntd)
        cv2.imwrite(fntd,crop_img)
        



(335, 385, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00371_b1.jpg
(298, 244, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00371_b2.jpg
(70, 239, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00379_b1.jpg
(183, 259, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00003_b1.jpg
(179, 235, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00003_b2.jpg
(181, 232, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00003_b3.jpg
(187, 256, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00003_b4.jpg
(190, 109, 3)
/home/student/data_5000_project/data/image

(281, 232, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00148_b4.jpg
(288, 270, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00148_b5.jpg
(299, 222, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00148_b6.jpg
(258, 323, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00019_b1.jpg
(273, 354, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00019_b2.jpg
(210, 344, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00019_b3.jpg
(240, 290, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00019_b4.jpg
(127, 138, 3)
/home/student/data_5000_project/data/imag

(224, 249, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00015_b4.jpg
(272, 255, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00532_b1.jpg
(341, 195, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00249_b1.jpg
(269, 179, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00249_b2.jpg
(243, 202, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00361_b1.jpg
(213, 230, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00361_b2.jpg
(177, 224, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00589_b1.jpg
(207, 222, 3)
/home/student/data_5000_project/data/imag

(254, 184, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00418_b1.jpg
(203, 141, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00418_b2.jpg
(197, 273, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00094_b1.jpg
(247, 280, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00174_b1.jpg
(236, 138, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00025_b1.jpg
(123, 136, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00025_b2.jpg
(176, 77, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00001_b1.jpg
(203, 79, 3)
/home/student/data_5000_project/data/images

(158, 151, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00095_b1.jpg
(183, 268, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00417_b1.jpg
(158, 265, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00417_b2.jpg
(154, 166, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00417_b3.jpg
(184, 296, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00362_b1.jpg
(160, 331, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00362_b2.jpg
(212, 275, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00131_b1.jpg
(252, 213, 3)
/home/student/data_5000_project/data/imag

(209, 115, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00549_b1.jpg
(106, 189, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00549_b2.jpg
(169, 149, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00087_b1.jpg
(118, 121, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00342_b1.jpg
(333, 296, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00268_b1.jpg
(190, 241, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00239_b1.jpg
(217, 350, 3)
/home/student/data_5000_project/data/images/construction_types/processed/opus_incertum/selected_training/td_00526_b1.jpg
(152, 221, 3)
/home/student/data_5000_project/data/imag